# Revenue Vehicles for NTD agencies with no GTFS RT

* `dim_annual_ntd_agency_information`
* `dim_organizations`

[Slack thread](https://cal-itp.slack.com/archives/C02H6JUSS9L/p1688662549443129)

In [ ]:
import pandas as pd
from calitp_data_analysis.tables import tbls
from calitp_data_analysis.sql import to_snakecase
from siuba import *

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def dim_orgs()->pd.DataFrame:
    # https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.dim_organizations
    df = (tbls.mart_transit_database.dim_organizations() 
        >> filter(_._is_current == True)
        >> select(_.ntd_id, _.name, 
                  _.gtfs_static_status, _.gtfs_realtime_status)
        >> collect()
        >> distinct()
       )    
    
    df = df[df.ntd_id.notna()].reset_index(drop=True)
    
    return df

In [ ]:
orgs = dim_orgs()

In [ ]:
def ntd_data(year: int) -> pd.DataFrame:
    
    df = (tbls.mart_ntd.dim_annual_ntd_agency_information() 
          >> filter(_.year == year, _.state == "CA")
          >> select(_.ntd_id, _.total_voms, _.agency_name, _.state)
          >> collect()
         )

    return df

In [ ]:
ntd = ntd_data(2021)

In [ ]:
df = pd.merge(
    orgs,
    ntd,
    on = "ntd_id",
    how = "outer",
    validate = "m:1",
    indicator=True
)

df._merge.value_counts()

In [ ]:
# Who hasn't merged on?
df[df._merge=="right_only"].agency_name.unique()

In [ ]:
no_rt_vehicles = df[df.gtfs_realtime_status == "RT Incomplete"].total_voms.sum()
yes_rt_vehicles = df[df.gtfs_realtime_status == "RT OK"].total_voms.sum()

In [ ]:
no_rt_vehicles / (no_rt_vehicles + yes_rt_vehicles)

In [ ]:
yes_rt_vehicles